In [4]:
#https://medium.com/@constandinou.antonio/quant-post-3-1-a-guided-path-into-mean-reversion-8b33b3c279e4
#https://medium.com/@bart.chr/pairs-trading-for-algorithmic-trading-breakdown-d8b709f59372

import pandas as pd
import statsmodels.api as sm
import numpy as np
import statsmodels.tsa.stattools as ts

In [5]:
df = pd.read_csv('datasets/data.csv')
data = df[['AALR3', 'ABCB4']]
data.columns = ['y', 'x']
x = sm.add_constant(data['x'])
y = data['y']

df = df[df.columns.difference(['Data'])]

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [6]:
# Fit and summarize OLS model
model = sm.OLS(y, x).fit()
print(model.params)

const   -6.936373
x        1.308303
dtype: float64


In [7]:
#https://groups.google.com/forum/#!topic/pystatsmodels/HbUIeSuXqZs
class DickeyFullerTest:
    def __init__(self):
        self.df = None
        self.crit = None
        self.slope = None
        self.halflife = None
        self.lookback = None
        self.coint = None
        
    def adf(self, y, x):

        model = sm.GLS(y,sm.add_constant(x)).fit()
        coint = model.resid

        print(model.params)

        adfstat, pvalue, critvalues, res = ts.adfuller(coint, store=True, regresults=True)

        self.df = model.params[1]/model.bse[1]
        self.crit = res.critvalues
        self.slope = model.params[1]
        self.halflife = -np.log(2)/model.params[1]
        self.lookback = int(round(-np.log(2)/model.params[1]))
        self.coint = coint
        
        print('df: ', self.df)
        #print('crit: ', self.crit)
        print('slope: ', self.slope)
        print('halflife: ', self.halflife)
        print('lookback: ', self.lookback)

In [8]:
test = DickeyFullerTest()
test.adf(y, data['x'])

const   -6.936373
x        1.308303
dtype: float64
df:  45.48603661109386
slope:  1.3083027670510512
halflife:  -0.5298064010995843
lookback:  -1


In [19]:
#https://medium.com/@bart.chr/pairs-trading-for-algorithmic-trading-breakdown-d8b709f59372
#https://github.com/aconstandinou/mean-reversion
class ADF(object):
    """
    Augmented Dickey–Fuller (ADF) unit root test
    """

    def __init__(self):
        self.p_value = None
        self.five_perc_stat = None
        self.perc_stat = None
        self.p_min = .0
        self.p_max = .05
        self.look_back = 63

    def apply_adf(self, time_series):
        model = ts.adfuller(time_series, 1)
        self.p_value = model[1]
        self.five_perc_stat = model[4]['5%']
        self.perc_stat = model[0]

    def use_P(self):
        return (self.p_value > self.p_min) and (self.p_value < self.p_max)
    
    def use_critical(self):
        return abs(self.perc_stat) > abs(self.five_perc_stat)

class Half_Life(object):
    """
    Half Life test from the Ornstein-Uhlenbeck process 
    """

    def __init__(self):
        self.hl_min = 1.0
        self.hl_max = 42.0
        self.look_back = 43
        self.half_life = None

    def apply_half_life(self, time_series):
        lag = np.roll(time_series, 1)
        lag[0] = 0
        ret = time_series - lag
        ret[0] = 0

        # adds intercept terms to X variable for regression
        lag2 = sm.add_constant(lag)

        model = sm.OLS(ret, lag2)
        res = model.fit()
        
        #self.half_life = round(-np.log(2) / res.params[1], 0)
        self.half_life = -np.log(2) / res.params[1]

    def use(self):
        return (self.half_life < self.hl_max) and (self.half_life > self.hl_min)

# a value > 0.5 indicates a trending time series. The greater the value above 0.5 the more trending it is.
# a value = 0.5 indicates a random walk.
# a value < 0.5 indicates a mean reverting time series. The closer the value gets to 0 the more mean reverting it is
class Hurst():
    """
    If Hurst Exponent is under the 0.5 value of a random walk, then the series is mean reverting
    """

    def __init__(self):
        self.h_min = 0.0
        self.h_max = 0.4
        self.look_back = 126
        #https://robotwealth.com/demystifying-the-hurst-exponent-part-1/
        self.lag_max = 20#era 100
        self.h_value = None
    
    def apply_hurst(self, time_series):
        lags = range(2, self.lag_max)

        tau = [np.sqrt(np.std(np.subtract(time_series[lag:], time_series[:-lag]))) for lag in lags]

        #poly = np.polyfit(np.log10(lags), np.log10(tau), 1)
        poly = np.polyfit(np.log(lags), np.log(tau), 1)

        self.h_value = poly[0]*2.0 

    def use(self):
        return (self.h_value < self.h_max) and (self.h_value > self.h_min)
    
def hurst_ernie_chan(p):
    lags = range(2, 20)
    variancetau = []; tau = []
    for lag in lags: 
        #  Write the different lags into a vector to compute a set of tau or lags
        tau.append(lag)

        # Compute the log returns on all days, then compute the variance on the difference in log returns
        # call this pp or the price difference
        pp = np.subtract(p[lag:], p[:-lag])
        variancetau.append(np.var(pp))

    # we now have a set of tau or lags and a corresponding set of variances.
    # plot the log of those variance against the log of tau and get the slope
    m = np.polyfit(np.log10(tau),np.log10(variancetau),1)

    hurst = m[0] / 2

    return hurst

In [11]:
life = Half_Life()
life.apply_half_life(model.resid)
print(life.use())
print(life.half_life)

True
3.4749378345843525


In [12]:
adf = ADF()
adf.apply_adf(model.resid)
print(adf.use_P())
print(adf.use_critical())

True
True


In [16]:
hurst = Hurst()
hurst.apply_hurst(model.resid.as_matrix())
print(hurst.use())
print(hurst.h_value)

True
0.2547481732950415


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
